In [1]:
import csv
import re
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter
from string import punctuation

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alessandrodimaria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alessandrodimaria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/alessandrodimaria/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
data_path = './../../../data'
advocate_dir = os.path.join(data_path, 'BeerAdvocate')
reviews_ba = pd.read_csv(os.path.join(advocate_dir, 'ratings_BA_clean.csv'))

rev_true_ba = reviews_ba[reviews_ba['review']==True]

In [ ]:
users = pd.read_csv(os.path.join(advocate_dir, 'users.csv'))
beer_BA = pd.read_csv(os.path.join(advocate_dir, 'beers.csv'))

In [5]:
beer_BA_grouped = beer_BA.groupby('style')
most_fam_b_per_style = pd.DataFrame()
for name, gruop in beer_BA_grouped:
    g = gruop.sort_values(ascending=[False],by=['nbr_reviews'])
    most_fam_b_per_style = pd.concat([most_fam_b_per_style, g.head(1)])

most_fam_stout = most_fam_b_per_style.sort_values(ascending=[False],by=['nbr_reviews']).iloc[1]
reviews_Stout = reviews_ba[reviews_ba['beer_id']==most_fam_stout.beer_id]
text_rev_stout = reviews_Stout[reviews_Stout['review']==True]['text']

In [3]:
expert_terms = {
    "flavor": ['caramel', 'toffee', 'bready', 'biscuity', 'nutty', 'roasted', 'chocolate', 'coffee', 'mocha','molasses','syrupy','smokey','toasted','vanilla','cocoa','brown sugar',
              'hoppy', 'citrusy', 'tropical', 'floral', 'piney', 'resinous', 'herbal', 'grassy', 'earthy', 'dank', 'spicy', 'peppery', 'juicy', 'zesty', 'tangy', 'sharp', 'resin', 'bitterness',
               'estery',' fruity', 'berry',' cherry',' apple',' banana', 'pear', 'stone fruit', 'plum', 'fig', 'raisin', 'peach',' apricot', 'dark fruit', 'citrus zest', 'lemon', 'orange peel', 'pineapple', 'mango', 'lychee',
               'oaky', 'woody', 'bourbon', 'whiskey', 'rye', 'tequila', 'brandy','vinous', 'sour', 'tart', 'acidic', 'lacto', 'brett', 'farmhouse', 'honeyed', 'clove', 'bubblegum', 'funky', 'barnyard'], 
    "aroma": ['bouquet', 'nose', 'aromatic', 'fragrant','perfumed', 'faint', 'musky', 'dank', 'subtle', 'fruity', 'floral', 'yeasty', 'clean', 'malty', 'crisp', 'pungent', 'spicy', 'smokey', 'earthy'],  
    "mouthfeel": [ 'body','full-bodied', 'medium-bodied', 'light-bodied', 'thick', 'thin', 'mouthfeel', 'creamy', 'smooth', 'velvety', 'oily', 'astringent','tannic', 'chalky', 'drying', 'slick', 'watery', 'effervescent', 'fizzy', 'tingly', 'prickly', 'carbonation', 'viscosity', 'warming', 'coating', 'biting', 'sharp'],
    "brewing": [ 'dry hopping', 'double dry hopping', 'barrel-aged', 'bottle-conditioned', 'open fermentation', 'secondary fermentation', 'wort', 'mash, sparging', 'cold crashing', 'decoction', 'conditioning', 'lacto', 'yeast strain', 'adjuncts', 'grains', 'malt', 'specialty grains'],
    "technical": [ 'balance', 'complexity', 'depth', 'layers', 'nuanced', 'refined', 'structured', 'profile', 'round', 'harmonious', 'clean', 'crisp', 'finish', 'lingering', 'evolving', 'sharp', 'clarity', 'purity', 'dense', 'robust', 'powerful', 'light', 'restrained', 'vibrant', 'subdued', 'heavy', 'integrity', 'layered', 'exemplar', 'benchmark', 'classic', 'signature style', 'finesse', 'elegance'],
    "appearance": ['hazy', 'cloudy', 'opaque', 'translucent', 'clear', 'bright', 'unfiltered', 'filtered', 'lacing', 'foam', 'frothy', 'rocky head', 'stable head', 'head retention', 'appearance', 'golden', 'amber', 'ruby', 'copper', 'dark', 'black', 'mahogany', 'light', 'straw', 'yellow', 'chestnut', 'tan', 'off-white head' ],
    "judgment": [ 'exemplar', 'well-integrated', 'restrained', 'harmonious', 'round', 'classic example', 'benchmark', 'flawless', 'exceptional', 'outstanding', 'world-class', 'traditional', 'innovative', 'unconventional', 'unique', 'reference point', 'nuanced' ],
    "off_flavors": [ 'oxidized', 'metallic', 'cardboard', 'stale', 'sulfur', 'skunky', 'diacetyl', 'astringent', 'phenolic', 'acetaldehyde', 'DMS', 'cloying', 'solvent-like', 'overly bitter', 'thin', 'harsh', 'vegetal', 'buttery', 'musty, moldy', 'medicinal' ],
    "miscellaneous": [ 'initial impression', 'mid-palate', 'aftertaste', 'finish', 'lingering', 'mouth-coating', 'evolving flavor', 'balanced start', 'unfolding', 'developing', 'peak', 'bright finish', 'dry finish', 'clean ending', 'reminiscent', 'similar to', 'akin to', 'comparable', 'surpasses', 'diverges from', 'evokes', 'hints of', 'resembles', 'distinct from', 'notes of', 'echoes' ]
}

In [4]:
expert_terms_stemmed = {}
for category, terms in expert_terms.items():
    tokens = [word_tokenize(term.lower()) for term in terms]
    
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word[0]) for word in tokens]
    expert_terms_stemmed[category] =  stemmed_tokens



In [11]:
exp_stem_set = []
for category, terms in expert_terms_stemmed.items():
    exp_stem_set = exp_stem_set + terms
    
exp_stem_set = set(exp_stem_set)

In [12]:
def extract_keywords(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    punkt_symbols = set(punctuation)
    removable_words= stop_words.union(punkt_symbols)

    filtered_tokens = [word for word in tokens if ((word not in removable_words) and word.isalpha())]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    return stemmed_tokens

In [13]:
def exp_stem_extract(text, exp_stem_set):
    extracted_keywords = set(extract_keywords(text))
    return list(exp_stem_set.intersection(extracted_keywords))

In [14]:
def stem_df(x, exp_stem_set):
    x = x.copy()
    x.loc[:, 'text_lower'] = x['text'].str.lower()
    stems =  x['text_lower'].apply(lambda x: exp_stem_extract(x,exp_stem_set))
    x = x.drop(columns=['text', 'text_lower'])
    return pd.concat([x, stems], axis=1).rename(columns={'text_lower':'stems'})

In [15]:
text_rev_stout = reviews_Stout[reviews_Stout['review']==True][['user_id', 'text']]


NameError: name 'reviews_Stout' is not defined

In [14]:
import time

start_time = time.time()

# Concatenate the results back with the original DataFrame
text_rev_stout = stem_df(text_rev_stout, exp_stem_set)

print("--- %s seconds ---" % (time.time() - start_time))

--- 2.859100341796875 seconds ---


In [15]:
text_rev_stout

,user_id,stems
3820003,mdaniel723.1205050,[coffe]
3820012,cmurphy7.1176491,"[coffe, bodi, vanilla, sharp, aftertast, finis..."
3820015,jprince103.1152897,"[coffe, chocol, overli, note]"
3820016,docdeens.1196859,[balanc]
3820026,openstoutlover.1183725,[]
...,...,...
3826851,jake_ramrod.719193,"[coffe, depth, slick, head, earthi, dark, hint..."
3826852,qwest_35.716659,"[coffe, bitter, mocha, complex, roast, dark, w..."
3826853,michaelnickerson.724553,"[coffe, bitter, head, smooth, initi, finish, d..."
3826854,jgargano.723897,"[balanc, initi, mouthfeel]"


In [16]:
start_time = time.time()

rev_true_ba = stem_df(rev_true_ba, exp_stem_set)

print("--- %s seconds ---" % (time.time() - start_time))

--- 1778.5225338935852 seconds ---


In [17]:
rev_true_ba.to_pickle(os.path.join(advocate_dir, 'reviews_with_exp_stems.pkl'))

In [34]:
len(rev_true_ba)

NameError: name 'rev_true_ba' is not defined

In [26]:
rev_true_ba[rev_true_ba['lang_tag']!='en']['stems']

119                             [crisp]
121        [head, crisp, creami, clean]
126                                  []
128                                  []
130                                  []
                       ...             
4812581                              []
4812590                         [hoppi]
4812600                              []
4812644                              []
4812670                           [rye]
Name: stems, Length: 126810, dtype: object

In [ ]:
data_path = './../../../data'
rb_dir = os.path.join(data_path, 'RateBeer')
reviews_rb = pd.read_csv(os.path.join(rb_dir, 'ratings_RB_clean.csv'))

rev_true_rb = reviews_rb[reviews_rb['review']==True]

In [4]:
len(rev_true_rb)

7121601

In [5]:
rev_true_rb = rev_true_rb[rev_true_rb['lang_tag']=='en']

In [6]:
len(rev_true_rb)

6606948

In [16]:
import time 
start_time = time.time()

rev_true_rb = stem_df(rev_true_rb, exp_stem_set)

print("--- %s seconds ---" % (time.time() - start_time))

--- 2529.523526906967 seconds ---


In [17]:
rev_true_rb.to_pickle(os.path.join(rb_dir, 'reviews_with_exp_stems.pkl'))

In [ ]:
len(rev_true_rb)